# Yield Curve Calibration

In this notebook we demonstrate yield curve calibration procedures.

Curve calibration is the process of deriving discount factor curves $P(tT)$ from quotations of financial instruments traded in the market. The financial instruments need to take into account various details and market conventions. These details and market conventions can be handled e.g. with QuantLib. This is why we use QuantLib for our curve calibration example.

The notebook is structured as follows:

  1. Single-curve calibration from swap quotes.

  2. Multi-curve calibration of OIS discount and Euribor projection curves.


Market instrument quotes are typically in terms of *market rates* or *par rates*. We read example data from data files.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import QuantLib as ql

EURSTR overnight swap quotes:

In [ ]:
eur_str_swaps_data = pd.read_csv('../data/eur_str_swaps.csv')
eur_str_swaps_data

Forward rate agreement quotes:

In [ ]:
euribor_6m_fras_data = pd.read_csv('../data/euribor_6m_fras.csv')
euribor_6m_fras_data

In [ ]:
euribor_3m_fras_data = pd.read_csv('../data/euribor_3m_fras.csv')
euribor_3m_fras_data

Vanilla swap quotes:

In [ ]:
euribor_6m_swaps_data = pd.read_csv('../data/euribor_6m_swaps.csv')
euribor_6m_swaps_data

Basis swap quotes:

In [ ]:
euribor_3m_6m_swaps_data = pd.read_csv('../data/euribor_3m_6m_swaps.csv')
euribor_3m_6m_swaps_data

## Single Curve Calibration

In a first step we illustrate single curve calibration. We use 6m swap quotes for this exercise.

Irrespective of single-curve or multi-curve calibration we first need to specify the interest rate indices and *rate helpers*.

In [ ]:
euribor_6m_bootstrap_index = ql.Euribor6M()

In [ ]:
swap_std_helpers = [
    ql.SwapRateHelper(
        row['Quote'],                  # rate
        ql.Period(row['Term']),        # tenor
        ql.TARGET(),                   # calender
        ql.Annual,                     # fixedFrequency
        ql.ModifiedFollowing,          # fixedConvention
        ql.Thirty360(ql.Thirty360.BondBasis),  # fixedDayCount
        euribor_6m_bootstrap_index     # index
        )
    for idx, row in euribor_6m_swaps_data.iterrows()
]

QuantLib exports various combinations of interpolation traits and interpolation methods:

| QuantLib Swig class          | Trait       | Method
|------------------------------|-------------|--------------------
| PiecewiseFlatForward         | ForwardRate | BackwardFlat
| PiecewiseLinearForward       | ForwardRate | Linear
| PiecewiseLinearZero          | ZeroYield   | Linear
| PiecewiseCubicZero           | ZeroYield   | Cubic
| PiecewiseKrugerZero          | ZeroYield   | Kruger
| PiecewiseConvexMonotoneZero  | ZeroYield   | ConvexMonotone
| PiecewiseLogLinearDiscount   | Discount    | LogLinear
| PiecewiseLogCubicDiscount    | Discount    | MonotonicLogCubic
| PiecewiseSplineCubicDiscount | Discount    | SplineCubic
| PiecewiseKrugerLogDiscount   | Discount    | KrugerLog

We use *PiecewiseFlatForward*, that is backward-flat interpolation on continuous forward rates.

In [ ]:
yts = ql.PiecewiseFlatForward(
    0,
    ql.TARGET(),
    swap_std_helpers,
    ql.Actual365Fixed()
)
yts.nodes()

Now, we can check that curve calibration and interpolation works as expected.

In [ ]:
times = np.linspace(0.0, 30.0, 301)
plt.figure(figsize=(8,5))
plt.plot(
    times,
    [ yts.forwardRate(T,T,ql.Continuous,ql.Annual,True).rate() * 100 for T in times ]
)
plt.xlabel('maturity time (in years)')
plt.ylabel('forward rate (%)')
plt.show()

## Multi-curve Calibration

Multi-curve calibration in QuantLib is analogous to single curve calibration. Critical aspect is the specification of rate helpers. The dependencies between curves are realised by supplying earlier calibrated curves to the rate helpers.

### OIS Curve

Standard OIS Curve calibration is like single-curve calbration: projection curve and discount curve coincide.

In [ ]:
eur_str_ytsh = ql.RelinkableYieldTermStructureHandle()
eur_str_index = ql.Eonia(eur_str_ytsh)  # Eonia and EURSTR shae the same properties
ql_quote = lambda q : ql.QuoteHandle(ql.SimpleQuote(q))

ois_rate_helpers = [
    ql.OISRateHelper(
        2,                             # settlementDays
        ql.Period(row['Term']),        # tenor
        ql_quote(row['Quote']),        # rate
        eur_str_index,                 # index
    )
    for idx, row in eur_str_swaps_data.iterrows()
]

yts = ql.PiecewiseFlatForward(
    0,
    ql.TARGET(),
    ois_rate_helpers,
    ql.Actual365Fixed()
)
display(yts.nodes())

eur_str_ytsh.linkTo(yts)

### Projection Curve from FRA and Vanilla Swap Quotes

Next we build the 6m projection curve.

In [ ]:
euribor_6m_ytsh = ql.RelinkableYieldTermStructureHandle()
euribor_6m_index = ql.Euribor6M(euribor_6m_ytsh)

euribor_6m_fra_helpers = [
    ql.FraRateHelper(
        ql_quote(row['Quote']),  # rate
        int(row['Term'][:1]),    # monthsToStart
        euribor_6m_index,        # index
    )
    for idx, row in euribor_6m_fras_data.iterrows()
]

euribor_6m_swap_helpers = [
    ql.SwapRateHelper(
        row['Quote'],                  # rate
        ql.Period(row['Term']),        # tenor
        ql.TARGET(),                   # calender
        ql.Annual,                     # fixedFrequency
        ql.ModifiedFollowing,          # fixedConvention
        ql.Thirty360(ql.Thirty360.BondBasis),  # fixedDayCount
        euribor_6m_bootstrap_index,    # index
        ql.QuoteHandle(),              # spread, not used
        ql.Period(),                   # fwdStart, not used
        eur_str_ytsh,                  # discountingCurve
        )
    for idx, row in euribor_6m_swaps_data.iterrows()    
]

yts = ql.PiecewiseFlatForward(
    0,
    ql.TARGET(),
    euribor_6m_fra_helpers + euribor_6m_swap_helpers,
    ql.Actual365Fixed()
)
display(yts.nodes())

euribor_6m_ytsh.linkTo(yts)

### Projection Curve from FRA and Basis Swap Quotes

We can also use basis swap helpers to construct a projection curve.

In [ ]:
euribor_3m_ytsh = ql.RelinkableYieldTermStructureHandle()
euribor_3m_index = ql.Euribor3M(euribor_3m_ytsh)

euribor_3m_fra_helpers = [
    ql.FraRateHelper(
        ql_quote(row['Quote']),  # rate
        int(row['Term'][:1]),    # monthsToStart
        euribor_3m_index,        # index
    )
    for idx, row in euribor_3m_fras_data.iterrows()
]

euribor_3m_6m_swap_helpers = [
    ql.IborIborBasisSwapRateHelper( # pay baseIndex + basis vs. otherIndex
        ql_quote(row['Quote']),        # basis
        ql.Period(row['Term']),        # tenor
        2,                             # settlementDays
        ql.TARGET(),                   # calendar
        ql.ModifiedFollowing,          # convention
        False,                         # endOfMonth
        euribor_3m_index,              # baseIndex
        euribor_6m_index,              # otherIndex
        eur_str_ytsh,                  # discountHandle
        True,                          # bootstrapBaseCurve
    )
    for idx, row in euribor_3m_6m_swaps_data.iterrows()
]

yts = ql.PiecewiseFlatForward(
    0,
    ql.TARGET(),
    euribor_3m_fra_helpers + euribor_3m_6m_swap_helpers,
    ql.Actual365Fixed()
)
display(yts.nodes())

euribor_3m_ytsh.linkTo(yts)

### Yield Curve Comparison

In [ ]:
times = np.linspace(0.0, 10.0, 301)
plt.figure(figsize=(8,5))
plt.plot(
    times,
    [ euribor_6m_ytsh.forwardRate(T,T,ql.Continuous,ql.Annual,True).rate() * 100 for T in times ],
    label = 'euribor_6m'
)
plt.plot(
    times,
    [ euribor_3m_ytsh.forwardRate(T,T,ql.Continuous,ql.Annual,True).rate() * 100 for T in times ],
    label = 'euribor_3m'
)
plt.plot(
    times,
    [ eur_str_ytsh.forwardRate(T,T,ql.Continuous,ql.Annual,True).rate() * 100 for T in times ],
    label = 'eur_str'
)
plt.legend()
plt.xlabel('maturity time (in years)')
plt.ylabel('forward rate (%)')
plt.show()